In [7]:
import requests
import codecs
import pickle
import numpy as np

In [26]:
from recommender_systems.extra_functions.work_with_matrices import *

# Создание нового пользователя и получение токена

In [27]:
URL_NEW_USER = 'http://127.0.0.1:5000/registration'

In [28]:
result = requests.post(URL_NEW_USER)
token = result.json()['token'] if result.status_code < 300 else f'{result.status_code}: {result.json()}'
token

'lfRuvoxcWXG4ttqKMerrGw'

In [29]:
headers = {
    'token': token
}

# Создание новой модели

In [30]:
URL_CREATE = 'http://127.0.0.1:5000/create'

In [31]:
sgd_params_create = {
    'dimension': 30, 
    'learning_rate': 0.0001, 
    'user_regularization': 0.2, 
    'item_regularization': 0.3
}

In [32]:
model_type = 'latent_factor_sgd_model'

In [33]:
params_create = {
    'type': model_type,
    'params': sgd_params_create
}

In [34]:
result = requests.post(URL_CREATE, json=params_create, headers=headers)
model_id = result.json()['id'] if result.status_code < 300 else f'{result.status_code}: {result.json()}'
model_id

8

# Обучение модели

In [50]:
URL_TRAIN = 'http://127.0.0.1:5000/train/'

In [51]:
PATH_TO_DATA = '../data_train/matrices/random_50_100_matrix.npz'

In [52]:
matrix = read_matrix_from_file(PATH_TO_DATA)
train_matrix = get_train_matrix(matrix, 0.3)

In [53]:
embedding_not_train_params = {'epochs_': 100}

In [54]:
embedding_train_params = {'epochs': 100}

In [55]:
params_not_train = {
    'params': embedding_not_train_params,
    'train_data': codecs.encode(pickle.dumps(train_matrix), 'base64').decode()
}

In [56]:
params_train = {
    'params': embedding_train_params,
    'train_data': codecs.encode(pickle.dumps(train_matrix), 'base64').decode()
}

In [57]:
result = requests.post(URL_TRAIN + str(model_id), json=params_not_train, headers=headers)
result_print = 'OK' if result.status_code < 300 else f'{result.status_code}: {result.json()}'
result_print

"400: {'message': 'Error in training'}"

In [58]:
result = requests.post(URL_TRAIN + str(model_id), json=params_train, headers=headers)
result_print = 'OK' if result.status_code < 300 else f'{result.status_code}: {result.json()}'
result_print

'OK'

# Получение статуса модели

In [59]:
URL_STATUS = 'http://127.0.0.1:5000/status/'

In [60]:
result = requests.post(URL_STATUS + str(model_id), headers=headers)
result_print = result.json()['status'] if result.status_code < 300 else f'{result.status_code}: {result.json()}'
result_print

'TRAINING'

In [61]:
result = requests.post(URL_STATUS + str(model_id), headers=headers)
result_print = result.json()['status'] if result.status_code < 300 else f'{result.status_code}: {result.json()}'
result_print

'READY'

# Получение предсказания модели

In [62]:
URL_ITEMS = 'http://127.0.0.1:5000/predict_items/'
URL_RATINGS = 'http://127.0.0.1:5000/predict_ratings/'

In [63]:
params_predict = {
    'user': 13
}

In [64]:
result = requests.get(URL_ITEMS + str(model_id), json=params_predict, headers=headers)
result_print = result.json()['result'] if result.status_code < 300 else f'{result.status_code}: {result.json()}'
np.array(result_print)

array([ 3, 11, 62, 30, 45, 98, 36, 25, 21, 99, 16, 60,  1, 20, 91, 75, 18,
       65,  0, 84, 74, 34, 44, 95, 10, 68, 88, 72, 48, 78, 35, 66,  5, 53,
       79,  7, 31, 15, 28, 67, 55, 82, 26, 47, 17, 24, 85, 58, 43, 86,  2,
       87, 19, 90, 57,  6, 23, 41, 12, 94, 42, 51, 76, 89, 29, 92, 39, 97,
       27, 56, 59, 22, 73, 46, 81, 71, 32, 40,  9,  8, 93, 61, 50,  4, 54,
       69, 37, 96, 63, 70, 38, 77, 80, 49, 13, 33, 83, 64, 14, 52])

In [65]:
result = requests.get(URL_RATINGS + str(model_id), json=params_predict, headers=headers)
result_print = result.json()['result'] if result.status_code < 300 else f'{result.status_code}: {result.json()}'
np.array(result_print)

array([ 2.05623204,  2.64544554,  0.17578107,  7.68479815, -2.27118021,
        0.95531113, -0.1678444 ,  0.81793449, -1.79698299, -1.79079713,
        1.4573744 ,  6.65557074, -0.28327849, -2.80964942, -3.43955458,
        0.75285188,  2.68684471,  0.49525324,  2.18613465, -0.08930122,
        2.59295795,  3.04785878, -1.22342341, -0.22919998,  0.3840099 ,
        3.67518086,  0.51187011, -1.12158544,  0.7423655 , -0.72490768,
        4.09072354,  0.78271238, -1.74633444, -2.82072303,  1.81364991,
        1.0715593 ,  3.92254401, -2.34790755, -2.52263511, -0.97105993,
       -1.77119644, -0.24188855, -0.4256759 ,  0.24897264,  1.55429171,
        3.97011897, -1.45266598,  0.50820935,  1.11445958, -2.76995864,
       -2.21441323, -0.48723941, -5.57569355,  0.9416595 , -2.30629956,
        0.57484548, -1.13443403, -0.1608947 ,  0.2689804 , -1.15129806,
        2.68636711, -2.20963661,  4.76503937, -2.43045436, -3.0892826 ,
        2.14409726,  1.00524431,  0.61183593,  1.44895298, -2.31

# Дообучение модели

In [66]:
result = requests.post(URL_TRAIN + str(model_id), json=params_train, headers=headers)
result_print = 'OK' if result.status_code < 300 else f'{result.status_code}: {result.json()}'
result_print

'OK'

# Изменение модели

In [67]:
URL_CHANGE = 'http://127.0.0.1:5000/change/'

In [68]:
sgd_params_change = {
    'dimension': 35, 
    'learning_rate': 0.0001, 
    'user_regularization': 0.2, 
    'item_regularization': 0.3
}

In [69]:
params_not_change = {
    'params': sgd_params_create
}

In [70]:
params_change = {
    'params': sgd_params_change
}

In [71]:
result = requests.post(URL_CHANGE + str(model_id), json=params_not_change, headers=headers)
result_print = 'OK' if result.status_code < 300 else f'{result.status_code}: {result.json()}'
result_print

"405: {'message': 'The parameters are the same. Use clear handle'}"

In [72]:
result = requests.post(URL_CHANGE + str(model_id), json=params_change, headers=headers)
result_print = 'OK' if result.status_code < 300 else f'{result.status_code}: {result.json()}'
result_print

'OK'

# Очищение модели

In [73]:
URL_CLEAR = 'http://127.0.0.1:5000/clear/'

In [74]:
result = requests.post(URL_CLEAR + str(model_id), headers=headers)
result_print = 'OK' if result.status_code < 300 else f'{result.status_code}: {result.json()}'
result_print

'OK'

In [75]:
result = requests.get(URL_ITEMS + str(model_id), json=params_predict, headers=headers)
result_print = result.json()['result'] if result.status_code < 300 else f'{result.status_code}: {result.json()}'
result_print

"405: {'message': 'Model should be trained'}"

# Удаление модели

In [76]:
URL_DELETE = 'http://127.0.0.1:5000/delete/'

In [77]:
result = requests.delete(URL_DELETE + str(model_id), headers=headers)
result_print = 'OK' if result.status_code < 300 else f'{result.status_code}: {result.json()}'
result_print

'OK'

In [78]:
result = requests.get(URL_ITEMS + str(model_id), json=params_predict, headers=headers)
result_print = result.json()['result'] if result.status_code < 300 else f'{result.status_code}: {result.json()}'
result_print

"405: {'message': 'No access to this model'}"

# Неправильные запросы

In [79]:
result = requests.delete(URL_DELETE + str(model_id))
result.json()

{'message': 'Token is not valid'}

In [80]:
result = requests.delete(URL_DELETE + str(1), headers=headers)
result.json()

{'message': 'No access to this model'}

In [81]:
result = requests.post(URL_CREATE, json=params_change, headers=headers)
result.json()

{'message': 'No needed parameters'}